In [33]:

import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from PIL import Image, ImageDraw, ImageFont
import os

In [35]:
# --- PATHS ---
MODEL_PATH = r"C:\Users\HP\Desktop\RP\RP\Model Training\Jeran\action.h5"
FONT_PATH = r"C:\Windows\Fonts\Nirmala.ttf" # Fix: Cleaned up the space before '='

In [37]:
# Load Model
print("Loading Model...")
model = load_model(MODEL_PATH)

Loading Model...


In [38]:
# Load Font
try:
    # Font size increased from 40 to 50 (Removed problematic Unicode character from comment)
    font = ImageFont.truetype(FONT_PATH, 50)
except:
    print("Warning: Nirmala Font not found. Using default.")
    font = ImageFont.load_default()


In [41]:
actions = np.array([
    'Beautiful', 'Black', 'Drink', 'Eat', 'Five',
    'Good_morning', 'Good_night', 'Hello',
    'Love', 'Mother', 'One', 'Run', 'Sleep', 'Talk', 'Ten',
    'Three', 'Today', 'Tomorrow', 'Two', 'Yesterday'
])

translate = {
    'Beautiful': {'en': 'Beautiful', 'ta': 'அழகு'},
    'Black': {'en': 'Black', 'ta': 'கருப்பு'},
    'Drink': {'en': 'Drink', 'ta': 'குடி'},
    'Eat': {'en': 'Eat', 'ta': 'சாப்பிடு'},
    'Five': {'en': 'Five', 'ta': 'ஐந்து'},
    'Good_morning': {'en': 'Good morning', 'ta': 'காலை வணக்கம்'},
    'Good_night': {'en': 'Good night', 'ta': 'இனிய இரவு'},
    'Hello': {'en': 'Hello', 'ta': 'வணக்கம்'},
    'Love': {'en': 'Love', 'ta': 'காதல்'},
    'Mother': {'en': 'Mother', 'ta': 'அம்மா'},
    'One': {'en': 'One', 'ta': 'ஒன்று'},
    'Run': {'en': 'Run', 'ta': 'ஓடு'},
    'Sleep': {'en': 'Sleep', 'ta': 'தூங்கு'},
    'Talk': {'en': 'Talk', 'ta': 'பேசு'},
    'Ten': {'en': 'Ten', 'ta': 'பத்து'},
    'Three': {'en': 'Three', 'ta': 'மூன்று'},
    'Today': {'en': 'Today', 'ta': 'இன்று'},
    'Tomorrow': {'en': 'Tomorrow', 'ta': 'நாளை'},
    'Two': {'en': 'Two', 'ta': 'இரண்டு'},
    'Yesterday': {'en': 'Yesterday', 'ta': 'நேற்று'}

}


In [43]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [45]:
# Helper Function 1: Unicode Text Drawing
def draw_unicode_text(img, text, position, color):
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)
    draw.text(position, text, font=font, fill=color)
    return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

# Helper Function 2: Keypoint Extraction (258 features)
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])


In [47]:
s# MAIN LOOP STARTING HERE
sequence = []
cap = cv2.VideoCapture(0)

# Set Camera Resolution
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Create Resizable Window
cv2.namedWindow('SIGN SIGHT', cv2.WINDOW_NORMAL)
cv2.resizeWindow('SIGN SIGHT', 1000, 600)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    print("AI STARTING... DO AN ACTION!")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        # 1. Convert to RGB for MediaPipe processing
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # 2. Process results (Run the MediaPipe Holistic model)
        results = holistic.process(image_rgb)
        
        # 3. Use the original frame as our canvas (It is already BGR)
        image = frame 
        
        # Draw keypoint connections on the frame
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Logic for collecting keypoints
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        
        # Keep only the last 30 frames (time window for the sign)
        sequence = sequence[-30:] 

        # STATUS BAR
        cv2.rectangle(image, (0,0), (1280, 60), (0,0,0), -1) # Status Bar Height increased to 60
        image = draw_unicode_text(image, "Status: Camera ON | Waiting for Action...", (10, 5), (200, 200, 200)) # Status Text position adjusted

        # Prediction
        if len(sequence) == 30:
            prediction = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]
            action = actions[np.argmax(prediction)]
            confidence = np.max(prediction)

            # SHOW LIVE CONFIDENCE TEXT
            conf_text = f"Detected: {action} ({int(confidence*100)}%)"
            # Detected Text position adjusted
            image = draw_unicode_text(image, conf_text, (10, 70), (0, 255, 0)) 

            if confidence > 0.5:
                # English Translation Display
                image = draw_unicode_text(image, f"EN: {translate.get(action, {'en': 'Unknown'})['en']}", (30, 150), (0, 255, 0))
                # Tamil Translation Display
                image = draw_unicode_text(image, f"TA: {translate.get(action, {'ta': 'தெரியாதது'})['ta']}", (30, 220), (0, 255, 255))
                # Sinhala section removed

        cv2.imshow('SIGN SIGHT', image)
        
        # Break gracefully on 'q' press
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

AI STARTING... DO AN ACTION!


Loading Model...
AI STARTING... DO AN ACTION!
